In [1]:
!python --version

Python 3.12.11


In [2]:
# =======================
# SETUP JAX + BIG_VISION
# =======================

# 1) Kiểm tra GPU và clone repo
import os
if 'NVIDIA_PRODUCT_NAME' in os.environ:
  !nvidia-smi
import jax
jax.devices()

!git clone --quiet --branch=main --depth=1 https://github.com/google-research/big_vision

# 2) Cài đặt các thư viện từ requirements.txt
# Lưu ý: Thay đổi đường dẫn để khớp với cấu trúc thư mục của repo
!pip -q install -r big_vision/big_vision/requirements.txt

# 3) Cài đặt các thư viện cần thiết khác và chuyển thư mục
!pip -q install --upgrade "transformers>=4.44,<5" accelerate pillow gdown faiss-cpu --no-cache-dir --default-timeout=180
!pip3 -q install --no-cache-dir -U crcmod
%cd big_vision

# 4) Kiểm tra JAX dùng GPU ổn định
os.environ.pop("JAX_PLATFORMS", None)

import jax.numpy as jnp
print("JAX version:", jax.__version__)
print("JAX devices:", jax.devices())

assert any(d.platform == "gpu" for d in jax.devices()), "JAX không thấy GPU. Hãy đảm bảo bạn đã chọn Runtime Type = GPU."

x = jnp.ones((2048, 2048))
y = jnp.dot(x, x.T).block_until_ready()
print("✅ OK: JAX chạy GPU thành công.")

Thu Aug 28 04:13:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Cell 2: Tải dữ liệu và giải nén (sử dụng đường dẫn tuyệt đối)
import zipfile
import os
import shutil

# --- VUI LÒNG XÁC NHẬN CÁC ID TỆP DƯỚI ĐÂY ---
keyframe_file_id = '1vTp9bsGXIJCmJGK2ufYJ1m1ZpgDfyLJz'
map_file_id = '1ft2iRv1YBqLSOqClWHY1SXnF4p-xu4rQ'
data_dir = '/content/data'

# Tạo thư mục làm việc và tải tệp xuống
os.makedirs(data_dir, exist_ok=True)
print("Đang tải tệp tin từ Google Drive...")
!gdown --id $keyframe_file_id -O {data_dir}/Keyframes_L21.zip
!gdown --id $map_file_id -O {data_dir}/map-keyframes-aic25-b1.zip

# Giải nén
with zipfile.ZipFile(f'{data_dir}/Keyframes_L21.zip', 'r') as zip_ref:
    zip_ref.extractall(data_dir)
with zipfile.ZipFile(f'{data_dir}/map-keyframes-aic25-b1.zip', 'r') as zip_ref:
    zip_ref.extractall(data_dir)

# Tìm đường dẫn đến keyframes
keyframes_dir = os.path.join(data_dir, 'keyframes', 'L21_V001')
if not os.path.exists(keyframes_dir):
    raise FileNotFoundError(f"Thư mục keyframes không tồn tại: {keyframes_dir}. Vui lòng kiểm tra lại ID tệp.")
keyframes_files = sorted([f for f in os.listdir(keyframes_dir) if f.endswith('.jpg')])
print(f"Đã tìm thấy {len(keyframes_files)} keyframes.")

Đang tải tệp tin từ Google Drive...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1vTp9bsGXIJCmJGK2ufYJ1m1ZpgDfyLJz
From (redirected): https://drive.google.com/uc?id=1vTp9bsGXIJCmJGK2ufYJ1m1ZpgDfyLJz&confirm=t&uuid=ba78e327-54da-4b96-8829-2234200a6553
To: /content/data/Keyframes_L21.zip
100% 1.45G/1.45G [00:18<00:00, 80.2MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ft2iRv1YBqLSOqClWHY1SXnF4p-xu4rQ
To: /content/data/map-keyframes-aic25-b1.zip
100% 1.60M/1.60M [00:00<00:00, 4.20MB/s]
Đã tìm thấy 307 keyframes.


In [4]:
# Sửa lại các lệnh import
import ml_collections
import big_vision.models.proj.image_text.two_towers as model_mod
from big_vision.pp import builder as pp_builder
from big_vision.pp import ops_general
from big_vision.pp import ops_image
from big_vision.pp import ops_text
from big_vision.pp.proj.image_text import ops_naflex
from big_vision.pp.proj.paligemma import ops
from jax.experimental.pjit import pjit
from jax.sharding import Mesh
from jax.lax import with_sharding_constraint
import numpy as np # Thêm numpy

# --- CẤU HÌNH MÔ HÌNH NAFLEX ---
# Chọn biến thể So400m NaFlex (kích thước lớn, hiệu suất cao, xử lý ảnh không vuông)
VARIANT, RES, N_PATCHES = 'So400m/16', 'naflex', 256 # Hoặc 1024 tùy nhu cầu
PATCH_SIZE = 16
TXTVARIANT = 'So400m'
EMBDIM = 1152
SEQLEN = 64
CKPT = f'siglip2_{VARIANT.lower().replace("/", "")}_{RES}.npz'

print(f"Đang sử dụng mô hình: {VARIANT} NaFlex")

# Tải checkpoint từ Google Cloud Storage
!test -f /tmp/{CKPT} || gsutil cp gs://big_vision/siglip2/{CKPT} /tmp/

# Cấu hình mô hình NaFlex tương tự như trong file demo
model_cfg = ml_collections.ConfigDict(dict(
    image_model='proj.image_text.naflex_vit',
    image=dict(
        pool_type='map',
        nposemb=16, # Sửa lại nposemb thành 16
        posemb='learn_2d',
        variant='So400m' # Thêm dòng này để mô hình biết nó cần sử dụng biến thể So400m
    ),
    text_model='proj.image_text.text_transformer',
    text=dict(
        scan=True,
        variant=TXTVARIANT,
        vocab_size=256_000,
    ),
    out_dim=[None, EMBDIM],
    bias_init=-10,
))
model = model_mod.Model(**model_cfg)
params = model_mod.load(None, f'/tmp/{CKPT}', model_cfg)
print("Đã tải mô hình NaFlex thành công!")

# Định nghĩa preprocessor cho NaFlex (đảm bảo bảo toàn tỷ lệ khung hình)
pp_naflex = pp_builder.get_preprocess_fn('|'.join((
    'value_range(-1, 1)',
    f'resize_to_sequence({PATCH_SIZE}, {N_PATCHES}, outkey="image")',
    f'patchify({PATCH_SIZE}, key="image")',
    'flatten(["image"])',
    f'pad_to_shape(key="image/patches", shape=({N_PATCHES}, None))',
    f'pad_to_shape(key="image/type", shape=({N_PATCHES},))',
    f'pad_to_shape(key="image/yidx", shape=({N_PATCHES},))',
    f'pad_to_shape(key="image/xidx", shape=({N_PATCHES},))',
    'tuplify(["image/patches", "image/type", "image/yidx", "image/xidx"], "image")',
)))

# Định nghĩa preprocessor cho văn bản (sử dụng Gemma tokenizer)
pp_txt = pp_builder.get_preprocess_fn(f'lower(key="text")|tok(length={SEQLEN}, model="gemma", bos="no", eos="sticky", key="text")')

# Sử dụng pjit cho JAX để tận dụng tốt GPU (tùy chọn)
mesh = Mesh(jax.devices(), ('data',))
data_sharding = jax.sharding.PartitionSpec(('data',))
@pjit
def get_image_features_pjit(patches):
    # Sửa dòng này để sử dụng data_sharding
    return with_sharding_constraint(model.apply({'params': params}, patches, None)[0],
                                     jax.sharding.NamedSharding(mesh, data_sharding))
@pjit
def get_text_features_pjit(texts):
    # Sửa dòng này để sử dụng data_sharding
    return with_sharding_constraint(model.apply({'params': params}, None, texts)[1],
                                     jax.sharding.NamedSharding(mesh, data_sharding))

Đang sử dụng mô hình: So400m/16 NaFlex
Copying gs://big_vision/siglip2/siglip2_so400m16_naflex.npz...
| [1 files][  4.2 GiB/  4.2 GiB]   39.7 MiB/s                                   
Operation completed over 1 objects/4.2 GiB.                                      
Đã tải mô hình NaFlex thành công!


In [5]:
# Thêm các import cần thiết
import os
import numpy as np
from PIL import Image # Thêm dòng này để import lớp Image
import jax
import jax.numpy as jnp
from tqdm import tqdm # Import tqdm cho thanh tiến trình
import math
import gc

# === Cell 4 (đã sửa): Trích xuất SigLIP embeddings cho TOÀN BỘ ảnh, có resume ===

if "keyframes_files" in globals():
    frames = sorted([f for f in keyframes_files if f.lower().endswith((".jpg",".jpeg",".png"))])
else:
    # Lỗi nếu keyframes_dir không tồn tại
    if not os.path.exists(keyframes_dir):
        raise FileNotFoundError(f"Thư mục keyframes không tồn tại: {keyframes_dir}")
    frames = sorted([f for f in os.listdir(keyframes_dir) if f.lower().endswith((".jpg",".jpeg",".png"))])

N = len(frames)
assert N > 0, "Không tìm thấy ảnh trong keyframes_dir"
BATCH_SIZE = 32
print("Số ảnh:", N)

# Suy ra chiều D từ 1 ảnh
with jax.disable_jit():
    with Image.open(os.path.join(keyframes_dir, frames[0])).convert("RGB") as _im:
        img_np = np.array(_im)
        tmp = pp_naflex(dict(image=img_np))['image']
        tmp_jnp = tuple(jnp.array(x)[jnp.newaxis, ...] for x in tmp)
        D = model.apply({'params': params}, tmp_jnp, None)[0].shape[1]
print(f"Chiều của vector embedding: {D}")

# Chuẩn bị memmap (nếu đã có -> r+, nếu chưa -> w+)
memmap_path = "siglip_features.mmap"
mode = "r+" if os.path.exists(memmap_path) else "w+"
features = np.memmap(memmap_path, dtype="float32", mode=mode, shape=(N, D))

# Resume index (số batch đã hoàn thành)
idx_path = memmap_path + ".idx"
start_batch = 0
if os.path.exists(idx_path):
    try:
        with open(idx_path, "r") as f:
            start_batch = int(f.read().strip() or "0")
    except:
        start_batch = 0
max_batches = math.ceil(N / BATCH_SIZE)
start_batch = max(0, min(start_batch, max_batches))
print("Resume từ batch index:", start_batch, "/", max_batches)

# Vòng lặp batched
for bi in tqdm(range(start_batch, max_batches)):
    i = bi * BATCH_SIZE
    batch_files = frames[i:i + BATCH_SIZE]
    batch_imgs = []
    for f in batch_files:
        img_path = os.path.join(keyframes_dir, f)
        with Image.open(img_path).convert("RGB") as im:
            batch_imgs.append(np.array(im.copy()))

    inputs_pp = [pp_naflex(dict(image=img))['image'] for img in batch_imgs]
    inputs_jnp = tuple(jnp.array(x) for x in zip(*inputs_pp))

    feats = get_image_features_pjit(inputs_jnp)
    feats = feats / jnp.linalg.norm(feats, axis=-1, keepdims=True)
    vecs = np.asarray(feats)

    features[i:i + len(batch_files), :] = vecs
    features.flush()

    with open(idx_path, "w") as f:
        f.write(str(bi + 1))

    del batch_imgs, inputs_pp, inputs_jnp, feats, vecs
    gc.collect()

final = np.asarray(features)
np.save("siglip_features.npy", final)
np.save("siglip_frames_order.npy", np.array(frames))
print("Hoàn tất:", final.shape, "-> siglip_features.npy")

try:
    os.remove(idx_path)
except FileNotFoundError:
    pass

Số ảnh: 307
Chiều của vector embedding: 1152
Resume từ batch index: 0 / 10


100%|██████████| 10/10 [01:50<00:00, 11.01s/it]

Hoàn tất: (307, 1152) -> siglip_features.npy


In [6]:
import faiss
import pandas as pd

embedding_path = "siglip_features.npy"
image_embeddings = np.load(embedding_path).astype("float32")
print("Image embeddings shape:", image_embeddings.shape)

d = image_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(image_embeddings)
print(f"Đã thêm {index.ntotal} vector vào chỉ mục FAISS (dim={d})")

# Load CSV map
# Sửa đường dẫn file CSV
csv_file_path = "/content/data/map-keyframes/L21_V001.csv"
map_df = pd.read_csv(csv_file_path)
print("CSV columns:", list(map_df.columns))
print(f"Đã load {len(map_df)} dòng ánh xạ từ {csv_file_path}")

if "frame_idx" in map_df.columns:
    map_df["frame_idx"] = pd.to_numeric(map_df["frame_idx"], errors="coerce")
if "pts_time" in map_df.columns:
    map_df["pts_time"] = pd.to_numeric(map_df["pts_time"], errors="coerce")

if "frame_idx" in map_df.columns:
    map_df.set_index("frame_idx", inplace=True, drop=False)

Image embeddings shape: (307, 1152)
Đã thêm 307 vector vào chỉ mục FAISS (dim=1152)
CSV columns: ['n', 'pts_time', 'fps', 'frame_idx']
Đã load 307 dòng ánh xạ từ /content/data/map-keyframes/L21_V001.csv


In [7]:
import re
_num_pat = re.compile(r'(\d+)')
def extract_frame_idx_from_name(fname: str):
    matches = _num_pat.findall(os.path.splitext(os.path.basename(fname))[0])
    return int(matches[-1]) if matches else None

if os.path.exists("siglip_frames_order.npy"):
    frames = np.load("siglip_frames_order.npy", allow_pickle=True).tolist()
else:
    frames = sorted([f for f in os.listdir(keyframes_dir) if f.lower().endswith((".jpg",".jpeg",".png"))])

if len(frames) != index.ntotal:
    raise ValueError(f"Số frames ({len(frames)}) không khớp số vector trong index ({index.ntotal}). "
                     "Hãy đảm bảo Cell 3 và Cell 5 dùng cùng danh sách/thuật toán sort.")

if 'frame_idx' not in map_df.columns:
    raise ValueError("CSV không có cột 'frame_idx'. Các cột hiện có: " + ", ".join(map_df.columns))

map_df['frame_idx'] = pd.to_numeric(map_df['frame_idx'], errors='coerce')
if 'pts_time' in map_df.columns:
    map_df['pts_time'] = pd.to_numeric(map_df['pts_time'], errors='coerce')

idx2row = {int(r.frame_idx): r for _, r in map_df.dropna(subset=['frame_idx']).iterrows()}

def search_keyframes(query_text, k=10):
    with jax.disable_jit():
        t_inputs_np = np.array([pp_txt({'text': query_text})['text']])
        t_inputs_jnp = jnp.array(t_inputs_np)

        t_feats = get_text_features_pjit(t_inputs_jnp)

    t_vec = np.asarray(t_feats).squeeze()
    t_vec = t_vec / (np.linalg.norm(t_vec) + 1e-12)
    t_vec = np.ascontiguousarray(t_vec.astype("float32")).reshape(1, -1)

    k = min(k, index.ntotal)
    sims, indices = index.search(t_vec, k)

    results = []
    for rank, idx in enumerate(indices[0]):
        fname = frames[int(idx)]
        fidx = extract_frame_idx_from_name(fname)
        row = idx2row.get(fidx)

        pts = None
        if row is not None and 'pts_time' in row and pd.notnull(row['pts_time']):
            try: pts = float(row['pts_time'])
            except: pts = row['pts_time']

        results.append({
            "rank": rank + 1,
            "score_cosine": float(sims[0][rank]),
            "filename": fname,
            "filepath": os.path.join(keyframes_dir, fname),
            "frame_idx": fidx,
            "pts_time": pts,
        })
    return results

In [8]:
# Ví dụ truy vấn
query = str(input("Nhập truy vấn: "))
top_results = search_keyframes(query, k=10)

print(f"Tìm thấy {len(top_results)} kết quả cho truy vấn: '{query}'")
print("\nTop 5 kết quả:")
for r in top_results[:5]:
    ts_show = f"{r['pts_time']:.2f}s" if isinstance(r["pts_time"], (int, float)) else r["pts_time"]
    print(f"Top {r['rank']}: {r['filename']} | frame_idx={r['frame_idx']} | pts_time={ts_show} | cosine={r['score_cosine']:.4f}")

Nhập truy vấn: A medium-shot, eye-level photo of a shoreline with a makeshift fence of dead, broken branches and wooden posts in the foreground, with the calm sea and a pale sky in the background. The text "HTV9 HD" and a timestamp are visible in the top right corner.
Tìm thấy 10 kết quả cho truy vấn: 'A medium-shot, eye-level photo of a shoreline with a makeshift fence of dead, broken branches and wooden posts in the foreground, with the calm sea and a pale sky in the background. The text "HTV9 HD" and a timestamp are visible in the top right corner.'

Top 5 kết quả:
Top 1: 020.jpg | frame_idx=20 | pts_time=None | cosine=0.1403
Top 2: 029.jpg | frame_idx=29 | pts_time=None | cosine=0.1284
Top 3: 028.jpg | frame_idx=28 | pts_time=None | cosine=0.1095
Top 4: 045.jpg | frame_idx=45 | pts_time=None | cosine=0.1084
Top 5: 046.jpg | frame_idx=46 | pts_time=None | cosine=0.0991
